# EEG DataLoader

The dimensions of the training set are as follows: 4,500 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_train.

The dimensions of the testing set are as follows: 750 samples, 64 channels, and a time length of 795. This corresponds to 5 categories in y_test.

You can download it from this Google Drive link: [https://drive.google.com/drive/folders/1ykR-mn4d4KfFeeNrfR6UdtebsNRY8PU2?usp=sharing]. 
Please download the data and place it in your data_path at "./data."

In [14]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [15]:
data_path = './data/'

In [16]:
train_data = np.load(data_path + 'train_data.npy')
test_data = np.load(data_path + 'test_data.npy')
train_label = np.load(data_path + 'train_label.npy')
test_label = np.load(data_path + 'test_label.npy')

train_data_mean = np.mean(train_data)
train_data_std = np.std(train_data)
train_data_normalized = (train_data - train_data_mean) / train_data_std
test_data_normalized = (test_data - train_data_mean) / train_data_std

#To convert the data into PyTorch tensors
x_train_tensor = torch.Tensor(train_data_normalized)
y_train_tensor = torch.LongTensor(train_label)
x_test_tensor = torch.Tensor(test_data_normalized)
y_test_tensor = torch.LongTensor(test_label)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Setting GPU on your computer

In [18]:
train_dataset = TensorDataset(x_train_tensor.to(device), y_train_tensor.to(device)) # input data to Tensor dataloader
train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True, shuffle=True) #  Batch size refers to the number of data sample
test_dataset = TensorDataset(x_test_tensor.to(device), y_test_tensor.to(device))
test_loader = DataLoader(test_dataset, batch_size=64,  drop_last=True,shuffle=False)

# Build simple Deep learning model

In [19]:
class EEGAutoencoderClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EEGAutoencoderClassifier, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 795, 384), # Input dimention is 64 channel * 795 time point, and use 256 units for first NN layer
            nn.BatchNorm1d(384),
            nn.LeakyReLU(negative_slope=0.05), # Use LeakyReLu function for NN training 
            nn.Dropout(0.1),
            nn.Linear(384, 128), # 256 NN units to 192 units
            nn.BatchNorm1d(128),
            nn.LeakyReLU(negative_slope=0.05),
            nn.Dropout(0.1),
            nn.Linear(128, 64),#  192 NN units to 128 units
            nn.LeakyReLU(negative_slope=0.05)
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes), # num_classes is 5 (hello,” “help me,” “stop,” “thank you,” and “yes”)
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for multi-class classification
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.encoder(x)
        
        # import pdb;pdb.set_trace()
        x = self.classifier(x)
        return x

In [20]:
num_classes = 5 # setting final output class
model = EEGAutoencoderClassifier(num_classes).to(device) 
criterion = nn.NLLLoss() # Use NLLLoss function to optimize
optimizer = optim.Adam(model.parameters(), lr=0.001) # Setting parameters learning rate = 0.001

In [21]:
num_epochs = 25 # setting training epochs (Number of training iterations)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader: 
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/25, Loss: 1.613937497138977
Epoch 2/25, Loss: 1.5935986042022705
Epoch 3/25, Loss: 1.6593667268753052
Epoch 4/25, Loss: 1.6664403676986694
Epoch 5/25, Loss: 1.4505765438079834
Epoch 6/25, Loss: 1.3423374891281128
Epoch 7/25, Loss: 1.2788554430007935
Epoch 8/25, Loss: 1.3412023782730103
Epoch 9/25, Loss: 1.1424118280410767
Epoch 10/25, Loss: 1.0930392742156982
Epoch 11/25, Loss: 1.1201082468032837
Epoch 12/25, Loss: 0.8920886516571045
Epoch 13/25, Loss: 0.8363171815872192
Epoch 14/25, Loss: 0.8095013499259949
Epoch 15/25, Loss: 0.8875225186347961
Epoch 16/25, Loss: 0.6809893846511841
Epoch 17/25, Loss: 0.6083361506462097
Epoch 18/25, Loss: 0.6252530217170715
Epoch 19/25, Loss: 0.5314902663230896
Epoch 20/25, Loss: 0.5511147975921631
Epoch 21/25, Loss: 0.39309418201446533
Epoch 22/25, Loss: 0.59262615442276
Epoch 23/25, Loss: 0.6370108723640442
Epoch 24/25, Loss: 0.37889501452445984
Epoch 25/25, Loss: 0.3389667868614197


In [22]:
model.eval() # Evaluate your model
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 48.44%


This is to check what the best variables are to get the best accuracy.

In [27]:
for i in [0.05, 0.02, 0.01, 0.005, 0.001]:
    for e in [5, 10, 25, 50]:
        print(f"Learning Rate = {i} and epoch = {e}")
        num_classes = 5 # setting final output class
        model = EEGAutoencoderClassifier(num_classes).to(device) 
        criterion = nn.NLLLoss() # Use NLLLoss function to optimize
        optimizer = optim.Adam(model.parameters(), i) # Setting parameters learning rate
        
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


        for epoch in range(e):
            model.train()
            for data, labels in train_loader: 
                optimizer.zero_grad()
                outputs = model(data)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            print(f'Epoch {epoch + 1}/{e}, Loss: {loss.item()}')
            
        
        model.eval() # Evaluate your model
        correct = 0
        total = 0

        with torch.no_grad():
            for data, labels in test_loader:
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Test Accuracy: {accuracy * 100:.2f}%')

Learning Rate = 0.5 and epoch = 5
Epoch 1/5, Loss: 11.859838485717773
Epoch 2/5, Loss: 2.453038454055786
Epoch 3/5, Loss: 1.6721996068954468
Epoch 4/5, Loss: 1.8306666612625122
Epoch 5/5, Loss: 1.8097319602966309
Test Accuracy: 20.17%
Learning Rate = 0.5 and epoch = 10
Epoch 1/10, Loss: 2.411003828048706
Epoch 2/10, Loss: 1.965744137763977
Epoch 3/10, Loss: 2.110834836959839
Epoch 4/10, Loss: 2.019683361053467
Epoch 5/10, Loss: 1.6033015251159668
Epoch 6/10, Loss: 1.812899112701416
Epoch 7/10, Loss: 1.6511797904968262
Epoch 8/10, Loss: 1.6344441175460815
Epoch 9/10, Loss: 1.9293426275253296
Epoch 10/10, Loss: 1.7440887689590454
Test Accuracy: 19.32%
Learning Rate = 0.5 and epoch = 25
Epoch 1/25, Loss: 2.5346286296844482
Epoch 2/25, Loss: 1.7508553266525269
Epoch 3/25, Loss: 1.6801506280899048
Epoch 4/25, Loss: 1.6270456314086914
Epoch 5/25, Loss: 1.785644769668579
Epoch 6/25, Loss: 1.9702941179275513
Epoch 7/25, Loss: 1.6226811408996582
Epoch 8/25, Loss: 1.6961874961853027
Epoch 9/25, 